In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Build Model

In [34]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [35]:
# Step 2: Prepare the Dataset with ImageFolder
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [36]:
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/GenAI/IDV Dataset/data/train', transform=transform)
val_dataset = datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/GenAI/IDV Dataset/data/val_set', transform=transform)

In [37]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [38]:
# Step 3: Modify ResNet-50
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3) # for 3 classes

In [39]:
# Step 4: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

#Model Training

In [40]:
# Step 5: Training and Validation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 2  # Number of training epochs

for epoch in range(num_epochs):
    # Training
    model.train()
    train_correct = 0
    train_total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    print(f"Epoch {epoch+1}, Train Accuracy: {train_accuracy}%")

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}, Validation Accuracy: {val_accuracy}%")

Epoch 1, Train Accuracy: 64.0746500777605%
Epoch 1, Validation Accuracy: 85.80246913580247%
Epoch 2, Train Accuracy: 83.98133748055987%
Epoch 2, Validation Accuracy: 93.20987654320987%


In [14]:
'''
# Step 5: Testing the Model
test_correct = 0
test_total = 0
model.eval()

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy}%")


'''

'\n# Step 5: Testing the Model\ntest_correct = 0\ntest_total = 0\nmodel.eval()\n\nwith torch.no_grad():\n    for inputs, labels in test_loader:\n        inputs, labels = inputs.to(device), labels.to(device)\n        outputs = model(inputs)\n        _, predicted = torch.max(outputs.data, 1)\n        test_total += labels.size(0)\n        test_correct += (predicted == labels).sum().item()\n\ntest_accuracy = 100 * test_correct / test_total\nprint(f"Test Accuracy: {test_accuracy}%")\n\n# Saving the model\ntorch.save(model.state_dict(), \'model.pth\')\n'

In [41]:
# Saving the model
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/GenAI/IDVmodel.pth')

#Test the Model

In [42]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # Adjust this if you have a different number of classes

model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/GenAI/IDVmodel.pth'))
model.eval()  # Set the model to evaluation mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [43]:
#Image Preprocessing

from PIL import Image

def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

In [52]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the specified device

def predict(image_path, model):
    image = preprocess_image(image_path)
    image = image.to(device)  # Move to the same device as your model

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

# Example usage
image_path = '/content/drive/MyDrive/Colab Notebooks/GenAI/IDV Dataset/testData/images266.jpg'
class_idx = predict(image_path, model)

In [53]:
class_names = train_dataset.classes  # Assuming train_dataset is your ImageFolder dataset

predicted_class_name = class_names[class_idx]
print("Predicted Class:", predicted_class_name)

Predicted Class: train_dosai
